# How to import html tables into python

In [68]:
import pandas as pd
import os
import numpy as np

In [36]:
szr_txt_fname=os.path.join(metadata_dir,'seizurelist_FR_'+sub+'.txt')
txt_df=pd.read_csv(szr_txt_fname,header=None,skiprows=4,sep='\t',names=['OnsetTime','OffsetTime','OnsetSample','OffsetSample'])

In [37]:
txt_df.head()

,OnsetTime,OffsetTime,OnsetSample,OffsetSample
0,2004-03-20 09:45:19.558594,2004-03-20 09:46:15.503906,8990,37634
1,2004-03-23 06:01:29.267578,2004-03-23 06:02:47.626953,62089,102209
2,2004-03-24 16:55:59.025391,2004-03-24 16:58:20.482422,1080333,1152759
3,2004-03-24 20:05:01.419922,2004-03-24 20:07:00.570313,1357015,1418020
4,2004-03-25 06:03:53.251953,2004-03-25 06:05:52.744141,1319041,1380221


In [79]:
# Import table from html file which lists onset channels and subclinical seizures
sub='253'
sub='264'
metadata_dir='/Users/davidgroppe/PycharmProjects/SZR_ANT/EU_METADATA'
htable=pd.read_html(os.path.join(metadata_dir,'pat_FR_'+sub+'.html'))

In [80]:
#Clinical szrs
list_id=1
htable[list_id]

,#,classif.,onset,pattern,vigilance,origin
0,1:,CP,eeg: 26.01.'05 23:48:52.945313,rhythmic beta waves,unclear,"HR6,BRA1,BRA2,BRA3,BRA4,BRB1,BRB2,BRB3,BRB4,HR..."
1,2:,CP,eeg: 27.01.'05 11:06:21.800781,rhythmic beta waves,unclear,"BLA1,BLB1,BLB2"
2,3:,CP,eeg: 27.01.'05 19:12:30.074219,rhythmic sharp waves,sleep stage II,"TRA1,TRA2,TRA3,TRA4,BRA1,BRA2,BRA3,BRA4,BRB1,B..."
3,4:,UC,eeg: 27.01.'05 23:32:13.699219,rhythmic beta waves,sleep stage II,"BLC1,BLC2"
4,5:,UC,eeg: 27.01.'05 23:44:08.398438,rhythmic beta waves,sleep stage II,"BLC1,BLC2,BLC3,BLC4"
5,6:,UC,eeg: 27.01.'05 23:55:16.351563,repetitive spiking,sleep stage II,"TRA1,TRA2,TRA3,BRA3"
6,7:,CP,eeg: 28.01.'05 00:03:10.425781,rhythmic sharp waves,sleep stage II,"TRA2,BRA1,TRA1,TRA3,TRA4,HR4,HR5,HR6,HR7"
7,8:,CP,eeg: 28.01.'05 09:50:29.683594,repetitive spiking,sleep stage II,TRA3


In [81]:
# Figure out which onset channel is the most frequent
list_id=1
n_clin_szr=htable[list_id].shape[0]
soz_list=list()
for a in range(n_clin_szr):
    temp_list=htable[list_id].iloc[a,5].split(',')
    for chan in temp_list:
        soz_list.append(chan)

uniq_chans, uniq_ct=np.unique(soz_list,return_counts=True)
print(uniq_chans)
print(uniq_ct)
max_id=np.argmax(uniq_ct)
soz_chan_mode=uniq_chans[max_id]
print('The most frequent SOZ channel is %s (%d of %d) szrs' % 
      (soz_chan_mode,uniq_ct[max_id],n_clin_szr))

['BLA1' 'BLB1' 'BLB2' 'BLC1' 'BLC2' 'BLC3' 'BLC4' 'BRA1' 'BRA2' 'BRA3'
 'BRA4' 'BRB1' 'BRB2' 'BRB3' 'BRB4' 'HR1' 'HR10' 'HR2' 'HR3' 'HR4' 'HR5'
 'HR6' 'HR7' 'HR8' 'HR9' 'TRA1' 'TRA2' 'TRA3' 'TRA4']
[1 1 1 2 2 1 1 3 2 3 2 2 2 2 1 2 1 2 2 3 3 3 3 2 1 3 3 4 2]
The most frequent SOZ channel is TRA3 (4 of 8) szrs


In [82]:
# Add seizure onset channels to text file data frame
list_id=1
n_row=htable[list_id].shape[0]
for a in range(n_row):
    print(htable[list_id].iloc[a,2].split('eeg: ')[1])
    date_str=htable[list_id].iloc[a,2].split('eeg: ')[1]
    just_date, just_hr=date_str.split(' ')
    day, mm, yr=just_date.split('.')
    full_yr='20'+yr[1:] # PRAY THERE ARE NO 20th CENTURY PATIENTS
    new_date=full_yr+'-'+mm+'-'+day+' '+just_hr
#     print(new_date)
    row_id=txt_df[txt_df['OnsetTime']==new_date].index.tolist()
    print(htable[list_id].iloc[row_id,5])
    if len(row_id)==0:
        print('Error: could not find text file match for %s' % date_str)
    elif len(row_id)==1:
        #txt_df['OnsetChans'].iloc[row_id]=htable[list_id].iloc[a,5]
        txt_df.iloc[row_id,4]=htable[list_id].iloc[a,5]
    else:
        print('Error: found multiple file matches for %s' % date_str)

26.01.'05 23:48:52.945313
Series([], Name: origin, dtype: object)
Error: could not find text file match for 26.01.'05 23:48:52.945313
27.01.'05 11:06:21.800781
Series([], Name: origin, dtype: object)
Error: could not find text file match for 27.01.'05 11:06:21.800781
27.01.'05 19:12:30.074219
Series([], Name: origin, dtype: object)
Error: could not find text file match for 27.01.'05 19:12:30.074219
27.01.'05 23:32:13.699219
Series([], Name: origin, dtype: object)
Error: could not find text file match for 27.01.'05 23:32:13.699219
27.01.'05 23:44:08.398438
Series([], Name: origin, dtype: object)
Error: could not find text file match for 27.01.'05 23:44:08.398438
27.01.'05 23:55:16.351563
Series([], Name: origin, dtype: object)
Error: could not find text file match for 27.01.'05 23:55:16.351563
28.01.'05 00:03:10.425781
Series([], Name: origin, dtype: object)
Error: could not find text file match for 28.01.'05 00:03:10.425781
28.01.'05 09:50:29.683594
Series([], Name: origin, dtype: obje

In [66]:
print(txt_df)

                    OnsetTime                  OffsetTime  OnsetSample  \
0  2004-03-20 09:45:19.558594  2004-03-20 09:46:15.503906         8990   
1  2004-03-23 06:01:29.267578  2004-03-23 06:02:47.626953        62089   
2  2004-03-24 16:55:59.025391  2004-03-24 16:58:20.482422      1080333   
3  2004-03-24 20:05:01.419922  2004-03-24 20:07:00.570313      1357015   
4  2004-03-25 06:03:53.251953  2004-03-25 06:05:52.744141      1319041   
5  2004-03-25 09:26:29.599609  2004-03-25 09:28:05.669922       751411   
6  2004-03-26 10:47:24.744141  2004-03-26 10:48:48.042969      1130365   

   OffsetSample      OnsetChans  
0         37634       HRB2,HRC2  
1        102209            HRB2  
2       1152759       HRB2,HRC2  
3       1418020  HRB2,HRB3,HRC2  
4       1380221       HRB2,HRC2  
5        800599  HRB2,HRB3,HRC2  
6       1173014  HRB2,HRB3,HRC2  


In [24]:
# Output subclinical szr info to text file
list_id=2
n_row=htable[list_id].shape[0]
for a in range(n_row):
    print(htable[list_id].iloc[a,1]) # onset
    print(htable[list_id].iloc[a,2]) # offset

16.03.'04 01:43:02
01:43:15
16.03.'04 07:19:36
07:20:21
16.03.'04 11:53:54
11:54:02
16.03.'04 12:38:16
12:38:23
16.03.'04 18:27:12
18:27:43
17.03.'04 02:56:20
02:56:30
17.03.'04 04:30:15
04:30:30
17.03.'04 06:23:41
06:24:06
17.03.'04 12:11:02
12:11:44
17.03.'04 12:30:10
12:31:03
17.03.'04 12:39:06
12:39:10
17.03.'04 19:27:04
19:27:23
18.03.'04 00:57:14
00:57:34
18.03.'04 01:52:51
01:53:05
18.03.'04 02:23:49
02:24:14
18.03.'04 04:13:08
04:13:29
18.03.'04 12:07:43
12:08:17
18.03.'04 13:55:08
13:56:02
18.03.'04 14:25:01
14:25:15
18.03.'04 15:21:24
15:21:35
18.03.'04 15:21:58
15:22:10
18.03.'04 16:16:00
16:16:19
18.03.'04 16:40:50
16:41:17
18.03.'04 16:49:45
16:50:18
18.03.'04 16:55:18
16:55:34
18.03.'04 21:08:19
21:08:32
18.03.'04 21:17:24
21:17:40
18.03.'04 23:02:59
23:03:49
19.03.'04 05:51:23
05:51:45
19.03.'04 06:09:44
06:09:57
19.03.'04 07:14:10
07:14:31
19.03.'04 08:26:01
08:26:19
19.03.'04 13:52:09
13:52:32
19.03.'04 15:15:28
15:15:53
19.03.'04 17:45:45
17:46:00
19.03.'04 18:53:44
1

In [20]:
n_row

72

In [4]:
#Subclinical szrs
bro[2]

,#,onset,offset
0,1:,16.03.'04 01:43:02,01:43:15
1,2:,16.03.'04 07:19:36,07:20:21
2,3:,16.03.'04 11:53:54,11:54:02
3,4:,16.03.'04 12:38:16,12:38:23
4,5:,16.03.'04 18:27:12,18:27:43
5,6:,17.03.'04 02:56:20,02:56:30
6,7:,17.03.'04 04:30:15,04:30:30
7,8:,17.03.'04 06:23:41,06:24:06
8,9:,17.03.'04 12:11:02,12:11:44
9,10:,17.03.'04 12:30:10,12:31:03
